In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from openai import OpenAI

with open("drive/MyDrive/llm_csv_reader/openai.key") as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key)
print(client.models.list())  # should return available models if the key works


SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-5-codex', created=1757527818, object='model', owned_by='system'), Model(id='gpt-audio-2025-08-28', created=1756256146, object='model', owned_by='system'), Model(id='gpt-realtime', created=1756271701, object='model', owned_by='system'), Model(id='gpt-realtime-2025-08-28', created=1756271773, object='model', owned_by='system'), Model(id='gpt-audio', created=1756339249, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='mode

In [22]:
"""
Minimal demo: ask an OpenAI LLM questions about a CSV of football matches.

CSV expected columns (example):
date,home_team,away_team,home_goals,away_goals,competition

Usage:
1) Set OPENAI_API_KEY in your environment.
2) Put your CSV path in the main() call at the bottom.
3) Run: python basic_llm_csv.py
"""

import os
from typing import Optional, Dict, Any
import pandas as pd
from openai import OpenAI
import json

MODEL = "gpt-5"  # change to a model you have access to, e.g. "gpt-4.1-mini"

# -----------------------------
# 1) Load CSV
# -----------------------------
def load_matches(csv_path: str) -> pd.DataFrame:
    """Read the CSV and do light parsing."""
    df = pd.read_csv(csv_path)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    return df

# -----------------------------
# 2) (Optional) Filter with pandas
# -----------------------------
def filter_matches(
    df: pd.DataFrame,
    team: Optional[str] = None,
    competition: Optional[str] = None,
    year: Optional[int] = None,
) -> pd.DataFrame:
    """Very simple filters to keep the demo approachable."""
    out = df.copy()

    if team:
        mask = out["home_team"].str.contains(team, case=False, na=False) | \
               out["away_team"].str.contains(team, case=False, na=False)
        out = out[mask]

    if competition and "competition" in out.columns:
        out = out[out["competition"].str.contains(competition, case=False, na=False)]

    if year and "date" in out.columns:
        out = out[out["date"].dt.year == year]

    return out

# -----------------------------
# 2-alternative) Have the LLM suggest filters
# -----------------------------
def llm_suggest_filters(question: str, df_columns: list[str]) -> Dict[str, Any]:
  """
  Ask the LLM to extract {team, competition, year} from a free-text question.
  Returns a dict with keys: team, competition, year (or None if unknown).
  """
  # Only ask for fields that exist in the CSV
  want_team = "home_team" in df_columns and "away_team" in df_columns
  want_comp = "competition" in df_columns
  want_year = "date" in df_columns

  schema_note = {
      "type": "object",
      "properties": {
          "team": {"type": ["string", "null"]} if want_team else {"type": "null"},
          "competition": {"type": ["string", "null"]} if want_comp else {"type": "null"},
          "year": {"type": ["integer", "null"]} if want_year else {"type": "null"},
      },
      "required": ["team", "competition", "year"]
  }

  system = (
      "Extract filters from the user's question for a football matches CSV.\n"
      "Only output a single JSON object with keys: team, competition, year.\n"
      "Use null if unknown or not specified. Year must be an integer (e.g., 2024) or null."
  )

  msg_user = (
      "Question:\n"
      f"{question}\n\n"
      "Return JSON ONLY, no extra text. Example:\n"
      '{"team":"Barcelona","competition":"La Liga","year":2024}'
  )

  resp = client.chat.completions.create(
      model=MODEL,
      temperature=1,
      messages=[
          {"role": "system", "content": system},
          {"role": "user", "content": msg_user},
      ]
  )

  text = resp.choices[0].message.content.strip()
  try:
      data = json.loads(text)
  except Exception:
      # Safe fallback if model didn't return proper JSON
      data = {"team": None, "competition": None, "year": None}

  # Force to our shape/types
  out = {
      "team": (data.get("team") if isinstance(data.get("team"), str) and data.get("team").strip() else None),
      "competition": (data.get("competition") if isinstance(data.get("competition"), str) and data.get("competition").strip() else None),
      "year": (int(data.get("year")) if isinstance(data.get("year"), (int, str)) and str(data.get("year")).isdigit() else None),
  }
  # Respect missing columns
  if not want_team: out["team"] = None
  if not want_comp: out["competition"] = None
  if not want_year: out["year"] = None
  return out

def filter_matches_auto(df, question: str):
    """
    Uses the LLM to infer filters from the question, then applies your original filter_matches.
    """
    inferred = llm_suggest_filters(question, df.columns.tolist())
    return filter_matches(
        df,
        team=inferred["team"],
        competition=inferred["competition"],
        year=inferred["year"],
    )

# -----------------------------
# 3) Build a tiny prompt
# -----------------------------
def build_prompt(user_question: str, rows: pd.DataFrame, max_rows: int = 20) -> str:
    """
    Keep things simple: include up to N rows as CSV text.
    For large datasets, you’d filter first, then include just what you need.
    """
    snippet = rows.head(max_rows).to_csv(index=False)
    instructions = (
        "You are a helpful football analyst. Use ONLY the rows below to answer. "
        "If the answer is not in the rows, say you don't know.\n\n"
        "ROWS (CSV):\n"
    )
    return f"{instructions}{snippet}\nQUESTION: {user_question}"

# -----------------------------
# 4) Ask the model
# -----------------------------
def ask_llm(prompt: str) -> str:
    """One simple chat completion call."""
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=1,
        messages=[
            {"role": "system", "content": "Be concise and factual."},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message.content.strip()

# -----------------------------
# 5) Put it together
# -----------------------------
def ask_about_matches(
    csv_path: str,
    question: str,
    team: Optional[str] = None,
    competition: Optional[str] = None,
    year: Optional[int] = None,
    max_rows: int = 20,
) -> str:
    """
    One convenience function:
    - load CSV,
    - (optionally) filter,
    - build prompt,
    - ask model.
    """
    print('-'*20)
    print(f'Answering question: {question}')
    df = load_matches(csv_path)
    df_small = filter_matches(df, team=team, competition=competition, year=year)
    print('-'*20)
    print('Filtered DF')
    print(df_small)
    print('-'*20)
    print('Automatically filtered DF')
    df_small = filter_matches_auto(df, question)
    print(df_small)
    prompt = build_prompt(question, df_small, max_rows=max_rows)
    print('\n')
    return ask_llm(prompt)

# -----------------------------
# Example usage
# -----------------------------
def main():
    csv_path = "./drive/MyDrive/llm_csv_reader/matches.csv"  # <- replace with your file
    # Example 1: direct question, no filters
    print(ask_about_matches(csv_path, "List the three highest scoring matches."))

    # Example 2: filtered question
    print(ask_about_matches(
        csv_path,
        "Summarize Barcelona's 2024 league results in one paragraph.",
        team="Barcelona",
        competition="La Liga",
        year=2024,
        max_rows=40
    ))

    # Example 3: don't know!
    print(ask_about_matches(
        csv_path,
        "Summarize Barcelona's 2024 league results in one paragraph.",
        max_rows=40
    ))
if __name__ == "__main__":
    main()


--------------------
Answering question: List the three highest scoring matches.
--------------------
Filtered DF
        date        home_team          away_team  home_goals  away_goals  \
0 2024-08-12        Barcelona        Real Madrid           3           2   
1 2024-08-19        Liverpool            Chelsea           1           1   
2 2024-08-26    Bayern Munich  Borussia Dortmund           4           0   
3 2024-09-02         Juventus           AC Milan           2           2   
4 2024-09-09              PSG               Lyon           5           1   
5 2024-09-16  Manchester City            Arsenal           2           3   
6 2024-09-23  Atletico Madrid            Sevilla           0           0   
7 2024-09-30      Inter Milan             Napoli           1           2   
8 2024-10-07        Barcelona    Atletico Madrid           4           1   
9 2024-10-14      Real Madrid            Sevilla           2           0   

      competition  
0         La Liga  
1  Premie